In [ ]:
import contextily as cx
import pandas as pd
import geopandas

import seaborn as sns
sns.set_theme(style="whitegrid")

## Loading data

In [ ]:
lor_gdf = geopandas.read_file("lor_friedrichshain_kreuzberg.GeoJSON")
estimated_parking_spots_kfz_gdf = geopandas.read_file("estimated_parking_spots_kfz.geojson")
cartagger_gdf = geopandas.read_file('counted_cars_from_cartagger_project.geojson')

In [ ]:
# Imports
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


def get_osm_data(lor_gdf, tags, crs="EPSG:25833", location="Berlin, Germany"):
    # Load data
    try:
        lor_gdf = gpd.read_file(lor_gdf)
    except:
        print("Can't load LOR file")
    # Obtain coordinates of POIs
    poi = ox.geometries_from_place(location, tags=tags)
    poi_points = poi[poi.geom_type == "Point"]

    # Drop unnecessary (debatable) information
    poi = poi_points.dropna(axis=1)
    poi["name"] = poi_points["name"]

    # Project to the same coordinate system
    poi = poi.to_crs(crs)

    # We look for the intersection of all pois and our Bezirk
    poi_lor = gpd.sjoin(poi, lor_gdf, predicate="intersects")

    return poi_lor


def calc_parking_spots(poi_lor, cars, est_parking_spots, buffer_size=20):
    # Load data
    try:
        est_parking_spots = gpd.read_file(est_parking_spots)
    except:
        print("Can't load parking spots file")

    # Define a circular region around each poi
    poi_lor_buffered = poi_lor.copy()
    poi_lor_buffered["geometry"] = poi_lor["geometry"].buffer(buffer_size)
    poi_lor_buffered = poi_lor_buffered.drop(labels=["index_right"], axis=1)

    # Intersect parking spots and area
    intersect_parking = gpd.sjoin(poi_lor_buffered, est_parking_spots)
    # Intersect tagged cars and area
    intersect_cars = gpd.sjoin(poi_lor_buffered, cars[cars.in_motion == 0].to_crs(25833))
    

    # Calculate parking spots
    intersect_parking["capacity"] = intersect_parking["capacity"].apply(int)
    dfpivot = pd.pivot_table(
        intersect_parking,
        index="name_left",
        columns="capacity",
        aggfunc={"capacity": len},
    )
    dfpivot.columns = dfpivot.columns.droplevel()
    dfpivot.index.names = ["name"]
    parking_counts = dfpivot.sum(axis=1)
    parking_counts = parking_counts.to_frame("No. parking spots")
    
    # Calculate tagged cars
    #intersect_cars["capacity"] = intersect_cars["capacity"]
    dfpivot = pd.pivot_table(
        intersect_cars,
        index="name",
        columns="size",
        aggfunc={"size": len},
    )
    dfpivot.columns = dfpivot.columns.droplevel()
    dfpivot.index.names = ["name"]
    car_counts = dfpivot.sum(axis=1)
    car_counts = car_counts.to_frame("No. cars tagged")

    # Merge back with the main dataframe
    poi_lor_counts = poi_lor.merge(parking_counts.reset_index(), on="name", how="left")
    poi_lor_counts = poi_lor_counts.merge(car_counts.reset_index(), on="name", how="left")
    
    poi_lor_counts["No. parking spots"] = poi_lor_counts["No. parking spots"].fillna(0)
    poi_lor_counts["No. cars tagged"] = poi_lor_counts["No. cars tagged"].fillna(0)
    poi_lor_counts['parking_consumption'] = poi_lor_counts['No. cars tagged'] / poi_lor_counts['No. parking spots']
    poi_lor_counts['name'] = poi_lor_counts['name'].fillna('none')
    poi_lor_counts['parking_consumption'] = poi_lor_counts['parking_consumption'].fillna(0)
    poi_lor_counts['parking_consumption'] = np.nan_to_num(poi_lor_counts['parking_consumption'])
    poi_lor_counts.loc[poi_lor_counts['parking_consumption'] > 1000, 'parking_consumption'] = poi_lor_counts['No. cars tagged']
    
    
    #poi_lor_buffered["No. parking spots"] = poi_lor_counts["No. parking spots"].fillna(0)

    return poi_lor_counts, poi_lor_buffered

In [ ]:
# Usage:
poi_lor = get_osm_data(
    r"data-ambassadors-parking-project/data/raw/friedrichshain-kreuzberg"+"/lor_friedrichshain_kreuzberg.GeoJSON",
    {"amenity": True, "shops":True},
    )

In [ ]:
parking_counts, parking_counts_buffered = calc_parking_spots(
    poi_lor, cartagger_gdf,
    r"data-ambassadors-parking-project/data/datathon"+"/parking/estimated_parking_spots_kfz.geojson",
    40,
    )
parking_counts.head()

In [ ]:
parking_counts[parking_counts.amenity=='driving_school']

In [ ]:
grouped_mean = parking_counts.groupby('amenity').sum().sort_values('No. cars tagged', ascending=False) # No. parking spots
grouped_mean['parking_consumption'] = grouped_mean['No. cars tagged'] / grouped_mean['No. parking spots']
grouped_mean.head(10)

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.barplot(x="amenity", y="No. cars tagged", data=grouped_mean.reset_index().iloc[:7])

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.barplot(x="amenity", y="No. parking spots", data=grouped_mean.reset_index().iloc[:7])

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.barplot(x="amenity", y="parking_consumption", data=grouped_mean.reset_index().iloc[:7])

In [ ]:
parking_counts.describe()

In [ ]:
len(parking_counts.amenity.unique().tolist())

In [ ]:
parking_counts_buffered.explore(m=estimated_parking_spots_kfz_gdf.explore(m=cartagger_gdf.explore()))